In [5]:
from pathlib import Path
import pickle
# from model_trainer.main_multitask_v3 import main

f1_cut = 0.35
count_of_bad_elms = {}
count_of_all_elms = {}

exp_dir = Path("/global/homes/d/drsmith/scratch-ml/multi_256_v28")
# trials = sorted(exp_dir.glob("L3*r44*"))
trials = sorted(exp_dir.glob("L3*r4441*"))
for trial in trials:
    scores_dir = trial / "scores"
    score_files = sorted(scores_dir.glob("elm_wise_f1_scores_ep*.pkl"), reverse=True)
    if len(score_files) == 0:
        continue
    final_score_file = score_files[0]
    with open(final_score_file, "rb") as f:
        final_scores = pickle.load(f)
    if not 'train' in final_scores:
        continue
    train_scores = final_scores['train']
    for elm_id, elm_f1 in train_scores.items():
        if elm_id not in count_of_all_elms:
            count_of_all_elms[elm_id] = 0
            count_of_bad_elms[elm_id] = 0
        count_of_all_elms[elm_id] += 1
        if elm_f1 < f1_cut:
            count_of_bad_elms[elm_id] += 1
fraction_bad_elms = {
    elm_id: count_of_bad_elms[elm_id] / count_of_all_elms[elm_id] 
    for elm_id in count_of_all_elms
    if (count_of_all_elms[elm_id] >= 3) and count_of_bad_elms[elm_id]
}
count_of_bad_elms = dict(sorted(count_of_bad_elms.items(), key=lambda item: item[1], reverse=True))
count_of_all_elms = dict(sorted(count_of_all_elms.items(), key=lambda item: item[1], reverse=True))
fraction_bad_elms = dict(sorted(fraction_bad_elms.items(), key=lambda item: item[1], reverse=True))
bad_elms = [elm_id for elm_id in fraction_bad_elms if (fraction_bad_elms[elm_id] >= 0.2 and count_of_bad_elms[elm_id] >= 2)]
# dump results
with open(f"count_of_bad_elms_{exp_dir.name}_v2.pkl", "wb") as f:
    pickle.dump(
        {
            "bad_elms": count_of_bad_elms,
            "all_elms": count_of_all_elms,
            "fraction_bad_elms": fraction_bad_elms,
            "bad_elms": bad_elms,
        },
        f,
    )